In [0]:
import os, sys
import pandas

See instructions on PowerPoint slides under Resources at bit.ly/intro-to-pyspark to learn how to set this up

In [0]:
# storage_account_key = "-----"
# storage_container = 'linkedin-analysis'
# storage_account = 'kepracticestorage'
# folder = 'linkedin_analysis_data'

# dbutils.fs.mount(
#   source = "wasbs://" + storage_container + "@" + storage_account + ".blob.core.windows.net",
#   mount_point = "/mnt/" + folder,
#   extra_configs = {"fs.azure.account.key." + storage_account + ".blob.core.windows.net": storage_account_key})

Below we are importing the data from the Blob Container we just mounted to the Databricks workspace

In [0]:
# Set path so code will be dynamic if you change the folder
folder = 'linkedin_analysis_data'
path = 'dbfs:/mnt/' + folder + "/"

# Read in data files
industry_growth = spark.read.csv(path + "public_use-industry-employment-growth.csv", header=True, inferSchema=True)
skill_penetration = spark.read.csv(path + "public_use-skill-penetration.csv", header=True, inferSchema=True)
migration_country = spark.read.csv(path + "public_use-talent-migration-country.csv", header=True, inferSchema=True)
migration_industry = spark.read.csv(path + "public_use-talent-migration-industry.csv", header=True, inferSchema=True)
migration_skills = spark.read.csv(path + "public_use-talent-migration-skill.csv", header=True, inferSchema=True)

In [0]:
path

Out[4]: 'dbfs:/mnt/linkedin_analysis_data/'

Use **display()** to show formatted **top rows** of a dataset. **NOTE:** If your data set is big, using display() may crash your cluster! Do not use if your data is very large.

In [0]:
display(migration_industry)

country_code,country_name,wb_income,wb_region,isic_section_index,isic_section_name,industry_name,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018
ae,United Arab Emirates,High income,Middle East & North Africa,C,Manufacturing,Defense & Space,425.85,158.29,79.59,171.66
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Computer Hardware,108.23,368.7,167.25,113.82
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Computer Software,1083.56,884.25,560.81,455.86
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Computer Networking,496.92,433.76,96.56,239.65
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Internet,2017.52,1383.74,794.39,659.77
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Semiconductors,448.4,746.75,-285.56,148.29
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Telecommunications,672.53,696.57,401.12,309.69
ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,Law Practice,786.32,501.92,383.88,310.56
ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,Legal Services,743.06,703.01,504.49,458.16
ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,Management Consulting,1121.73,693.28,581.3,575.7


**Calculating Grouped Summaries:**
* **groupBy('variable1', 'variable2')** allows you to create a grouped summary for calculating mean, counts, median, etc.
* Follow groupBy() with .mean() in order to find a grouped average
* Also shown below is how to sort, ascending shown first and descending shown in the code block afterwards 
* Now, click on the bar graph icon below the table to see the data graphed!

In [0]:
# Calculating average net_per_10K_2018 grouped by isic_section_name sorted in ascending order
display(migration_industry.groupBy('isic_section_name').avg('net_per_10K_2018').orderBy('avg(net_per_10K_2018)'))

isic_section_name,avg(net_per_10K_2018)
Public administration and defence; compulsory social security,-65.82500000000003
Education,-57.81394422310754
Construction,-13.860051282051277
"Electricity, gas, steam and air conditioning supply",-9.392162162162169
Human health and social work activities,-6.2292361111111045
Accommodation and food service activities,3.857803030303029
Other service activities,9.988325791855214
Wholesale and retail trade; repair of motor vehicles and motorcycles,13.936296296296318
Information and communication,14.680044444444412
Mining and quarrying,17.7905487804878


In [0]:
from pyspark.sql.functions import col

# Calculating same average net_per_10K_2018 grouped summary by isic_section_name sorted in descending order
display(migration_industry.groupBy('isic_section_name').avg('net_per_10K_2018').orderBy(col('avg(net_per_10K_2018)').desc()))

isic_section_name,avg(net_per_10K_2018)
Not_mapped,83.9932142857143
Administrative and support service activities,34.61909090909089
"Arts, entertainment and recreation",30.41341013824886
"Agriculture, forestry and fishing",24.34470588235294
Professional scientific and technical activities,20.581992924528382
Real estate activities,20.370131578947362
Manufacturing,18.551971830985906
Financial and insurance activities,18.24072463768121
Transportation and storage,17.8335
Mining and quarrying,17.7905487804878


Shown below are the steps for calculating **grouped summaries using multiple columns and multiple metrics** such as averages and counts
* Keep the groupBy() syntax the same as it was above
* Put the grouped summary code inside the .agg() function
* Now you can use a specific grouped summary function like countDistinct below by:
  1. Importing the summary function you are using
  2. Use a summary function such as countDistinct('column_name') or avg('column_name') with the variable name to calculate the summary on inside it in quotes
  3. Add .alias("new_column_name") to change the column name in our output data set
  4. Separate multiple metrics by commas
  5. Make sure the entire text after .agg() is surrounded in parentheses

In [0]:
from pyspark.sql.functions import countDistinct, avg, max, min
from pyspark.sql import DataFrameStatFunctions as statFunc

# Finding grouped summary statistics by isic_section_name
display(migration_industry.groupBy('isic_section_name').agg(countDistinct('country_name').alias("country_count"), \
                                                            avg('net_per_10K_2018').alias('mean_net_per_10K_2018'), \
                                                            min('net_per_10K_2018').alias('min_net_per_10K_2018'), \
                                                            max('net_per_10K_2018').alias('max_net_per_10K_2018')) \
        .orderBy('mean_net_per_10K_2018'))

isic_section_name,country_count,mean_net_per_10K_2018,min_net_per_10K_2018,max_net_per_10K_2018
Public administration and defence; compulsory social security,97,-65.82499999999999,-4816.31,645.5
Education,115,-57.813944223107576,-2532.64,776.95
Construction,93,-13.860051282051273,-1949.51,711.84
"Electricity, gas, steam and air conditioning supply",37,-9.392162162162162,-845.47,287.83
Human health and social work activities,79,-6.229236111111108,-1279.0,521.74
Accommodation and food service activities,92,3.857803030303034,-1917.48,731.86
Other service activities,98,9.988325791855209,-905.4,785.88
Wholesale and retail trade; repair of motor vehicles and motorcycles,80,13.936296296296302,-1826.65,1169.05
Information and communication,119,14.680044444444452,-2187.68,1354.88
Mining and quarrying,108,17.790548780487807,-1380.05,768.14


In [0]:
summary = migration_industry.groupBy('isic_section_name').agg(countDistinct('country_name').alias("country_count"), \
                                                            avg('net_per_10K_2018').alias('mean_net_per_10K_2018'),
                                                            min('net_per_10K_2018').alias('min_net_per_10K_2018'),
                                                            max('net_per_10K_2018').alias('max_net_per_10K_2018')) \
        .orderBy('mean_net_per_10K_2018')

summary.columns

Out[12]: ['isic_section_name',
 'country_count',
 'mean_net_per_10K_2018',
 'min_net_per_10K_2018',
 'max_net_per_10K_2018']

In [0]:
# Good for small data sets
#display(skill_penetration)
# Show is better for large datasets
skill_penetration.show()

+----+--------------------+----------------+------------------+--------------------+--------------------+---------------------------+
year|skill_group_category|skill_group_name|isic_section_index| isic_section_name| industry_name|skill_group_penetration_pct|
+----+--------------------+----------------+------------------+--------------------+--------------------+---------------------------+
2015| Business Skills|Accounts Payable| M|Professional scie...| Accounting| 0.66|
2015| Business Skills|Accounts Payable| M|Professional scie...| Executive Office| 0.21|
2015| Business Skills|Accounts Payable| M|Professional scie...| Law Practice| 0.2|
2015| Business Skills|Accounts Payable| B|Mining and quarrying| Oil & Energy| 0.14|
2015| Business Skills|Accounts Payable| C| Manufacturing| Printing| 0.12|
2015| Business Skills|Accounts Payable| C| Manufacturing|Packaging & Conta...| 0.12|
2015| Business Skills|Accounts Payable| C| Manufacturing| Machinery| 0.11|
2015| Business Skills|Accounts Payable| B|Mining and quarrying| Mining & Metals| 0.08|
2015| Business Skills|Accounts Payable| M|Professional scie...|Environmental Ser...| 0.08|
2015| Business Skills|Accounts Payable| C| Manufacturing| Textiles| 0.08|
2015| Business Skills|Accounts Payable| C| Manufacturing|Electrical & Elec...| 0.07|
2015| Business Skills|Accounts Payable| C| Manufacturing| Food Production| 0.07|
2015| Business Skills|Accounts Payable| C| Manufacturing| Automotive| 0.07|
2015| Business Skills|Accounts Payable| M|Professional scie...|Mechanical Or Ind...| 0.07|
2015| Business Skills|Accounts Payable| C| Manufacturing|Industrial Automa...| 0.07|
2015| Business Skills|Accounts Payable| M|Professional scie...| Legal Services| 0.07|
2015| Business Skills|Accounts Payable| C| Manufacturing| Chemicals| 0.07|
2015| Business Skills|Accounts Payable| J|Information and c...|Information Techn...| 0.06|
2015| Business Skills|Accounts Payable| C| Manufacturing|Paper & Forest Pr...| 0.06|
2015| Business Skills|Accounts Payable| C| Manufacturing| Plastics| 0.06|
+----+--------------------+----------------+------------------+--------------------+--------------------+---------------------------+
only showing top 20 rows

-sandbox
##Guidance from Databricks

The Koalas project makes data scientists more productive when interacting with big data, by implementing the pandas DataFrame API on top of Apache Spark. By unifying the two ecosystems with a familiar API, Koalas offers a seamless transition between small and large data.

### [Performance](https://databricks.com/blog/2019/08/22/guest-blog-how-virgin-hyperloop-one-reduced-processing-time-from-hours-to-minutes-with-koalas.html)

<div style="img align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2019/08/koalas-image4.png" width="1000"/>
</div>

**Pandas** DataFrames are mutable, eagerily evaluated, and maintain row order. They are restricted to a single machine, and are very performant when the data sets are small, as shown in a).

**Spark** DataFrames are distributed, lazily evaluated, immutable, and do not maintain row order. They are very performant when working at scale, as shown in b) and c).

**Koalas** provides the best of both worlds: pandas API with the performance benefits of Spark. However, it is not as fast as implementing your solution natively in Spark, and let's see why below.

###Converting Data from Long to Wide Format with Koalas
**Example Where Koalas is Helpful:**
* Converting your data from long to wide and wide to long is difficult to do in PySpark. What can be done in one line in Pandas is done in many more in PySpark
* All of our data sets have a different column for each year except the above skill_penetration. We should convert skill_penetration to be a wide data set to be consistent with the other data
* This is a great example of when the koalas package is useful. Koalas allows you to use pandas code that is distributed in Spark
* First you convert to a koalas dataframe, then pandas code can be used, and then it can be converted back into a Spark dataframe

**Converting from Koalas and to Spark Dataframes and Back:**
- To convert from a Spark to a koalas data frame use df.to_koalas()
- To convert back to a Spark data frame from a koalas data frame use df.to_spark()

In [0]:
display(skill_penetration)

year,skill_group_category,skill_group_name,isic_section_index,isic_section_name,industry_name,skill_group_penetration_pct
2015,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Accounting,0.66
2015,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Executive Office,0.21
2015,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Law Practice,0.2
2015,Business Skills,Accounts Payable,B,Mining and quarrying,Oil & Energy,0.14
2015,Business Skills,Accounts Payable,C,Manufacturing,Printing,0.12
2015,Business Skills,Accounts Payable,C,Manufacturing,Packaging & Containers,0.12
2015,Business Skills,Accounts Payable,C,Manufacturing,Machinery,0.11
2015,Business Skills,Accounts Payable,B,Mining and quarrying,Mining & Metals,0.08
2015,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Environmental Services,0.08
2015,Business Skills,Accounts Payable,C,Manufacturing,Textiles,0.08


In [0]:
from databricks import koalas as ks

# Converting to Koalas data frame
skill_penetration_ks = skill_penetration.to_koalas()
# Converting data set from long to wide with pandas syntax
skill_penetration_wide = skill_penetration_ks.pivot_table(index=['skill_group_category', 'skill_group_name', 'isic_section_index', 'isic_section_name', 'industry_name'], \
            columns='year', values='skill_group_penetration_pct').reset_index().to_spark()

In [0]:
display(skill_penetration_wide)

skill_group_category,skill_group_name,isic_section_index,isic_section_name,industry_name,2015,2016,2017,2018
Specialized Industry Skills,Automotive,R,"Arts, entertainment and recreation",Sports,0.04,0.03,0.02,0.02
Soft Skills,Time Management,C,Manufacturing,Chemicals,0.05,0.06,0.1,0.14
Soft Skills,Leadership,M,Professional scientific and technical activities,Information Services,1.68,1.86,1.94,1.71
Business Skills,Product Marketing,R,"Arts, entertainment and recreation",Animation,0.08,0.08,0.11,0.22
Specialized Industry Skills,Research,M,Professional scientific and technical activities,Graphic Design,0.11,0.15,0.14,0.16
Specialized Industry Skills,Recruiting,B,Mining and quarrying,Mining & Metals,0.25,0.18,0.15,0.14
Specialized Industry Skills,Event Planning,C,Manufacturing,Electrical & Electronic Manufacturing,0.04,0.04,0.06,0.07
Specialized Industry Skills,Theatre,J,Information and communication,Broadcast Media,0.07,0.07,0.07,0.05
Business Skills,Product Marketing,J,Information and communication,Publishing,1.14,1.23,0.92,0.89
Business Skills,Manufacturing Operations,C,Manufacturing,Plastics,6.73,5.73,4.03,3.71


**Renaming Columns:** You rename each column one at a time in Spark with the function **withColumnRenamed('old_variable_name','new_variable_name')**. You can chain together these statements to change multiple variable names.

In [0]:
from pyspark.sql.functions import *

# Renaming columns created in conversion to a wide dataset above
skill_penetration_wide = skill_penetration_wide.withColumnRenamed("2015","skill_pen_pct_2015").withColumnRenamed("2016","skill_pen_pct_2016"). \
                                                          withColumnRenamed("2017","skill_pen_pct_2017").withColumnRenamed("2018","skill_pen_pct_2018")

**How to Print Column Names**

In [0]:
skill_penetration_wide.columns

Out[17]: ['skill_group_category',
 'skill_group_name',
 'isic_section_index',
 'isic_section_name',
 'industry_name',
 'skill_pen_pct_2015',
 'skill_pen_pct_2016',
 'skill_pen_pct_2017',
 'skill_pen_pct_2018']

In [0]:
display(migration_skills)

country_code,country_name,wb_income,wb_region,skill_group_category,skill_group_name,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Physical Security,-6015.99,-4128.4,-2594.15,-2304.27
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Public Safety,-4789.61,-2868.6,-2210.77,-1917.52
af,Afghanistan,Low income,South Asia,Tech Skills,Computer Networking,-4724.3,-2725.53,-3973.61,-1686.29
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Delivery Operations,-3164.92,-1891.85,-1289.91,-1673.86
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Army,-5423.62,-2925.31,-2311.54,-1570.09
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Air Force,-5754.7,-2833.44,-2249.2,-1489.44
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Nonprofit Management,-1242.03,-1705.95,-1600.58,-1346.22
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Politics,-1375.55,-1359.61,-1607.97,-1335.7
af,Afghanistan,Low income,South Asia,Specialized Industry Skills,Construction Engineering,-3286.46,-2475.74,-2297.12,-1255.01
af,Afghanistan,Low income,South Asia,Disruptive Tech Skills,Human Computer Interaction,-1931.24,-1192.98,-439.36,-1120.66


**Row Count by Group:** Below I calculated a row count by skill_group_category to see how many rows are in each unique category

In [0]:
migration_skills.groupBy('skill_group_category').count().show()

+--------------------+-----+
skill_group_category|count|
+--------------------+-----+
 Soft Skills| 1074|
Specialized Indus...|12015|
 Tech Skills| 2101|
 Business Skills| 4431|
Disruptive Tech S...| 1082|
+--------------------+-----+

In [0]:
display(migration_country)

base_country_code,base_country_name,base_lat,base_long,base_country_wb_income,base_country_wb_region,target_country_code,target_country_name,target_lat,target_long,target_country_wb_income,target_country_wb_region,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,af,Afghanistan,33.93911,67.709953,Low Income,South Asia,0.19,0.16,0.11,-0.05
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,am,Armenia,40.069099,45.038189,Upper Middle Income,Europe and Central Asia,0.1,0.05,0.03,-0.01
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,ao,Angola,-11.202692,17.873887,Lower Middle Income,Sub Saharan Africa,-0.01,0.04,0.11,-0.02
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,ar,Argentina,-38.416097,-63.616672,High Income,Latin America and the Caribbean,0.16,0.18,0.04,0.01
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,at,Austria,47.516231,14.550072,High Income,Europe and Central Asia,0.11,-0.08,-0.07,-0.05
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,au,Australia,-25.274398,133.775136,High Income,East Asia and Pacific,-1.06,-3.31,-4.01,-4.58
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,az,Azerbaijan,40.143105,47.576927,Upper Middle Income,Europe and Central Asia,0.24,0.25,0.1,0.05
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,bd,Bangladesh,23.684994,90.356331,Lower Middle Income,South Asia,-0.21,-0.21,-0.26,-0.18
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,be,Belgium,50.503887,4.469936,High Income,Europe and Central Asia,0.51,0.22,-0.01,-0.09
ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East and North Africa,bg,Bulgaria,42.733883,25.48583,Upper Middle Income,Europe and Central Asia,0.17,0.09,-0.07,-0.02


**Dropping columns:** We do not need the latitude and longitude variables for this analysis so I am showing how to drop those columns
Use syntax **df.drop('var1','var2','var3')** to drop columns

In [0]:
migration_country = migration_country.drop('base_lat','base_long','target_lat','target_long')
migration_country.columns

Out[18]: ['base_country_code',
 'base_country_name',
 'base_country_wb_income',
 'base_country_wb_region',
 'target_country_code',
 'target_country_name',
 'target_country_wb_income',
 'target_country_wb_region',
 'net_per_10K_2015',
 'net_per_10K_2016',
 'net_per_10K_2017',
 'net_per_10K_2018']

In [0]:
migration_country = migration_country.select('base_country_code','base_country_name','base_country_wb_income','base_country_wb_region','target_country_code', \
                                             'target_country_name','target_country_wb_income','target_country_wb_region','net_per_10K_2015', \
                                             'net_per_10K_2016','net_per_10K_2017','net_per_10K_2018')
migration_country.columns

Out[19]: ['base_country_code',
 'base_country_name',
 'base_country_wb_income',
 'base_country_wb_region',
 'target_country_code',
 'target_country_name',
 'target_country_wb_income',
 'target_country_wb_region',
 'net_per_10K_2015',
 'net_per_10K_2016',
 'net_per_10K_2017',
 'net_per_10K_2018']

**Finding grouped average migration for each country**: We have metrics in migration_country on how many individuals migrate out of each country into another. To merge this in with our other data files, we will need one row per country. Accordingly I am calculating the average net migration for each year migrating out of each country.

In [0]:
migration_country_avg = migration_country.groupBy('base_country_code','base_country_name','base_country_wb_income','base_country_wb_region'). \
                              agg(avg('net_per_10K_2015').alias('avg_country_mig_per_10K_2015'), \
                                  avg('net_per_10K_2016').alias('avg_country_mig_per_10K_2016'), \
                                  avg('net_per_10K_2017').alias('avg_country_mig_per_10K_2017'), \
                                  avg('net_per_10K_2018').alias('avg_country_mig_per_10K_2018'))

In [0]:
display(migration_country_avg)

base_country_code,base_country_name,base_country_wb_income,base_country_wb_region,avg_country_mig_per_10K_2015,avg_country_mig_per_10K_2016,avg_country_mig_per_10K_2017,avg_country_mig_per_10K_2018
lu,Luxembourg,High Income,Europe and Central Asia,11.059629629629633,10.097037037037035,8.37037037037037,8.503703703703703
md,Moldova,Lower Middle Income,Europe and Central Asia,-5.07875,-6.75875,-6.713750000000001,-6.302500000000001
se,Sweden,High Income,Europe and Central Asia,0.28312499999999996,0.38734374999999993,0.45906249999999993,0.4918750000000001
bo,Bolivia,Lower Middle Income,Latin America and the Caribbean,0.5680000000000001,0.26,-0.269,-0.38499999999999995
za,South Africa,Upper Middle Income,Sub Saharan Africa,0.45799999999999996,0.2031666666666667,-0.06833333333333334,-0.07350000000000005
pl,Poland,High Income,Europe and Central Asia,-0.11739130434782612,0.03195652173913045,0.105,0.10521739130434785
jp,Japan,High Income,East Asia and Pacific,0.04466666666666666,0.2426666666666667,0.16088888888888891,0.4664444444444444
kz,Kazakhstan,Upper Middle Income,Europe and Central Asia,0.555625,-0.18624999999999994,-0.835,-0.526875
sv,El Salvador,Lower Middle Income,Latin America and the Caribbean,0.4857142857142857,-0.63,-0.9628571428571429,-1.0728571428571432
al,Albania,Upper Middle Income,Europe and Central Asia,0.34142857142857136,0.042142857142857114,-1.3428571428571427,-0.36071428571428565


In [0]:
display(industry_growth)

country_code,country_name,wb_region,wb_income,isic_section_index,isic_section_name,industry_id,industry_name,growth_rate_2015,growth_rate_2016,growth_rate_2017,growth_rate_2018
ae,United Arab Emirates,Middle East & North Africa,High income,B,Mining and quarrying,56,Mining & Metals,7.89,5.57,3.99,0.16
ae,United Arab Emirates,Middle East & North Africa,High income,B,Mining and quarrying,57,Oil & Energy,3.14,0.92,0.52,0.67
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,15,Pharmaceuticals,15.88,11.59,7.19,2.93
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,23,Food Production,5.52,2.58,1.08,0.17
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,52,Aviation & Aerospace,10.58,7.13,4.39,2.45
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,53,Automotive,8.49,4.8,2.73,0.61
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,54,Chemicals,10.53,7.57,5.11,1.6
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,55,Machinery,9.76,6.99,3.03,2.38
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,58,Shipbuilding,4.76,2.75,-0.21,0.11
ae,United Arab Emirates,Middle East & North Africa,High income,C,Manufacturing,60,Textiles,12.0,8.47,4.53,-0.35


In [0]:
# industry_growth, migration_country_avg, migration_skills, skill_penetration_wide, migration_industry

In [0]:
#industry_growth: 'country_code','country_name','wb_region','wb_income','isic_section_index','isic_section_name','industry_id',
    #'industry_name','growth_rate_2015','growth_rate_2016','growth_rate_2017','growth_rate_2018'

#migration_country_avg: 'base_country_code','base_country_name','base_country_wb_income','base_country_wb_region','avg_country_mig_per_10K_2015',
    #'avg_country_mig_per_10K_2016','avg_country_mig_per_10K_2017','avg_country_mig_per_10K_2018'
  
#migration_skills: 'country_code','country_name','wb_income','wb_region','skill_group_category','skill_group_name','net_per_10K_2015',
    #'net_per_10K_2016','net_per_10K_2017','net_per_10K_2018'
  
#migration_industry: 'country_code','country_name','wb_income','wb_region','isic_section_index','isic_section_name','industry_name','net_per_10K_2015',
    #'net_per_10K_2016','net_per_10K_2017','net_per_10K_2018'
  
#skill_penetration_wide: 'skill_group_category','skill_group_name','isic_section_index','isic_section_name','industry_name','skill_pen_pct_2015',
    #'skill_pen_pct_2016','skill_pen_pct_2017','skill_pen_pct_2018'

**Thought process:** 

**Goal:** We want one row per country since we are most interested in looking at and predicting high and low country-to-country migration. 

**How to get there:** 
- We should join those data frames that summarize country and industry
- Then with the data frames that give more information with country and skill group penetration we can use the skill group and industry information to connect back to the country-level data frame. One example would be a categorical variable that identifies countries that have high skill penetration in the technology industry with a "high, medium, low" categorical variable.

See https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/ for a tutorial on joins in PySpark

In [0]:
industry_mig_growth = industry_growth.join(migration_industry, (industry_growth.country_code == migration_industry.country_code) & \
                                                               (industry_growth.country_name == migration_industry.country_name) & \
                                                               (industry_growth.wb_region == migration_industry.wb_region) & \
                                                               (industry_growth.wb_income == migration_industry.wb_income) & \
                                                               (industry_growth.isic_section_index == migration_industry.isic_section_index) & \
                                                               (industry_growth.isic_section_name == migration_industry.isic_section_name) & \
                                                               (industry_growth.industry_name == migration_industry.industry_name), how='outer')

In [0]:
industry_growth.count()

Out[29]: 7198

In [0]:
migration_industry.count()

Out[30]: 5282

In [0]:
industry_mig_growth.count()

Out[31]: 2833

**Count Number of Missing Values in a Column** added in each dataframe
Use syntax **df.where(df['column'].isNull()).count()**

Using the where function is one option for filtering data. The filter function shown below is the other option.

In [0]:
industry_mig_growth.where(industry_mig_growth['growth_rate_2018'].isNull()).count()

Out[30]: 2449

In [0]:
industry_mig_growth.where(industry_mig_growth['net_per_10K_2017'].isNull()).count()

Out[31]: 4365

**We have a missing data problem when merging data**
You can see above that when industry-level dataframes are merged together, there is a lot of missing data. Kept as is, this data is useless.

**Addressing Missing Data Problem**
Since there are high amounts of missing data in the industry data when merged (i.e. many countries do not have all industries recorded in the data), a new strategy is to sum together larger groups of industries we are interested in, which will merge into the country-level data.

In [0]:
# Listing the isic_section_name so I can find the one that will hold technology industries
display(migration_industry.select('isic_section_name').distinct())

isic_section_name
Human health and social work activities
Education
"Agriculture, forestry and fishing"
"Electricity, gas, steam and air conditioning supply"
Not_mapped
Accommodation and food service activities
"Arts, entertainment and recreation"
Transportation and storage
Administrative and support service activities
Construction


**Filtering**

Below you can see how to filter based on the values of a column. Here, we are subsetting the data to "Information and communication" industries so we can identify the technology industries within this larger group. I have a hypothesis that overall migration will be higher when technology migration is high.

Below in the code you can see how to **filter with one condition**

To **filter with multiple conditions**, use the operator & for "and" and | for "or"
df.filter(df.column_name1 == 'value1' | df.column_name1 == 'value2')
df.filter(df.column_name1 == 'value1' & df.column_name2 == 'value2')

In [0]:
# Filtering for isic_section_name equaling "Information and communication" to list out the industry names in this group
migration_tech_science = migration_industry.filter(migration_industry.isic_section_name == 'Information and communication')
display(migration_tech_science.select('industry_name').distinct())

industry_name
Broadcast Media
Online Media
Computer Hardware
Motion Pictures & Film
Newspapers
Wireless
Computer Networking
Publishing
Internet
Writing & Editing


Below I am creating a summary data set with the sum of each net_per_10k migration column across all technology industries by country

**Adding a new column to your current dataframe based on other values in your dataframe**
- Always use the **withColumn()** function to add a new column to your dataframe
- The first argument will always be the desired **new column name**
- Afterwards you should specify how that column should be built

**If/then statements**

Below we used the combination of **when** and **otherwise** to set a condition and return one value if the condition is true. The **otherwise()** function is added to specify the value if the condition is not true.

newdf = df.withColumn('new_column_name', when(condition, "value if condition is true").otherwise('value if condition is false'))

Below I also used an efficiency short cut and used isin() instead of specifying multiple conditions like col('col1') == 'value1' | col('col2') == 'value2' | col('col3') == 'value3' etc... Using isin is much less typing!

Place a list of the values to check for inside isin() and you are good to go!

In [0]:
from pyspark.sql.functions import *

# Creating a list of all technology industries from the list printed above
tech_industries = ['Computer Hardware','Wireless','Computer Networking','Internet','Computer Software','Telecommunications','Information Technology & Services', \
                   'Computer & Network Security','Computer Games']

# Adding a new identifier column called "tech_industry"
migration_industry = migration_industry.withColumn('tech_industry', when(col('industry_name').isin(tech_industries), 'Technology Industry').otherwise('Not a Technology Industry'))

In [0]:
display(migration_industry)

country_code,country_name,wb_income,wb_region,isic_section_index,isic_section_name,industry_name,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018,tech_industry
ae,United Arab Emirates,High income,Middle East & North Africa,C,Manufacturing,Defense & Space,425.85,158.29,79.59,171.66,Not a Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Computer Hardware,108.23,368.7,167.25,113.82,Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Computer Software,1083.56,884.25,560.81,455.86,Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Computer Networking,496.92,433.76,96.56,239.65,Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Internet,2017.52,1383.74,794.39,659.77,Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Semiconductors,448.4,746.75,-285.56,148.29,Not a Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,Telecommunications,672.53,696.57,401.12,309.69,Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,Law Practice,786.32,501.92,383.88,310.56,Not a Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,Legal Services,743.06,703.01,504.49,458.16,Not a Technology Industry
ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,Management Consulting,1121.73,693.28,581.3,575.7,Not a Technology Industry


Finally we are creating a new dataframe with the summed net migration numbers for only tech industries. We will merge these into the country-level data set

In [0]:
# Finally we are creating a new dataframe with the summed net migration numbers for only tech industries
# We will merge these into the country-level data set
tech_migration = migration_industry.groupBy('country_name','tech_industry').agg(sum('net_per_10k_2015').alias("tech_net_per_10k_2015"), \
                                                                                sum('net_per_10K_2016').alias('tech_net_per_10k_2016'), \
                                                                                sum('net_per_10K_2017').alias('tech_net_per_10k_2017'), \
                                                                                sum('net_per_10K_2018').alias('tech_net_per_10k_2018')) \
                                   .filter(migration_industry.tech_industry == 'Technology Industry').drop('tech_industry')

+------------------+--------------------+---------------------+---------------------+---------------------+---------------------+
 country_name| tech_industry|tech_net_per_10k_2015|tech_net_per_10k_2016|tech_net_per_10k_2017|tech_net_per_10k_2018|
+------------------+--------------------+---------------------+---------------------+---------------------+---------------------+
 Senegal| Technology Industry| 46.35| 100.96000000000001| -70.85| -216.82|
 Luxembourg|Not a Technology ...| 22631.770000000004| 22051.52| 19113.2| 19492.139999999996|
 Ethiopia| Technology Industry| 143.82| -31.74| -98.04| -102.22|
West Bank and Gaza|Not a Technology ...| -100.63| -626.95| -899.07| -1370.78|
 Portugal| Technology Industry| 56.040000000000006| 211.45| 853.72| 1194.89|
 France|Not a Technology ...| -1164.14| 2234.2600000000007| 5242.509999999998| 4398.039999999999|
 Israel| Technology Industry| 53.63| -126.32000000000002| 13.280000000000001| 1.7300000000000004|
 Serbia| Technology Industry| 111.80000000000001| -96.30000000000001| -337.76| -518.0799999999999|
 Austria| Technology Industry| 816.93| 852.3199999999999| 729.14| 461.79999999999995|
 Honduras|Not a Technology ...| 68.81| 20.15| -36.43| -87.47|
 Cameroon|Not a Technology ...| -156.49| -628.75| -1239.3| -1588.11|
 Uganda|Not a Technology ...| 1188.0099999999998| 480.61| 286.58| 235.88000000000002|
 Guatemala|Not a Technology ...| -135.35000000000002| -480.99999999999994| -87.81| 1.979999999999972|
 China|Not a Technology ...| 4994.350000000001| 1860.83| -1330.7400000000005| -923.1700000000003|
 Morocco|Not a Technology ...| 663.2500000000001| -4567.680000000001| -7416.819999999999| -9516.86|
 Romania| Technology Industry| -458.53999999999996| -516.51| -468.48| -602.03|
 Zimbabwe| Technology Industry| 117.74000000000001| -56.97| -216.69| -221.61|
 Bahamas, The|Not a Technology ...| 148.89| -52.91| 313.37| 193.07|
 Mozambique| Technology Industry| 145.59| 78.31| -129.32| -23.37|
 Sudan|Not a Technology ...| -1288.19| -1442.5| -1133.02| -1530.91|
+------------------+--------------------+---------------------+---------------------+---------------------+---------------------+
only showing top 20 rows

# Complete the following for extra practice. I will cover the solutions next class.

Calculate the average growth_rate_2018 by wb_income with the industry_growth dataframe

Calculate a grouped summary by wb_region with the average, minimum, and maximum growth_rate_2018, renaming each variable to new variables names using the alias() function
Sort the values in descending order

Rename net_per_10K_2015, net_per_10K_2016, net_per_10K_2017, and net_per_10K_2018 in migration_skills to net_skill_10K_2015, net_skill_10K_2016, etc.

Explore the select() and drop() functionality to subset your dataframes to a smaller number of columns.

## Challenge Question!

Create a grouped summary by industry_name and country finding the sum net_per_10k features. Filter the data to several isic_section_name values you are interested in exploring that may be associated with overall country talent migration. Use the method above where I used koalas to take that dataframe and turn it from long to wide format with each isic_section_name now having its own column for the sum of migration. We will later use this in our country-level data set which will hopefully have less missing data.